# Why use LCEL

We recommend reading the LCEL Get started section first.

LCEL makes it easy to build complex chains from basic components. It does this by providing: 1. A unified interface: Every LCEL object implements the Runnable interface, which defines a common set of invocation methods (invoke, batch, stream, ainvoke, …). This makes it possible for chains of LCEL objects to also automatically support these invocations. That is, every chain of LCEL objects is itself an LCEL object. 2. Composition primitives: LCEL provides a number of primitives that make it easy to compose chains, parallelize components, add fallbacks, dynamically configure chain internal, and more.

To better understand the value of LCEL, it’s helpful to see it in action and think about how we might recreate similar functionality without it. In this walkthrough we’ll do just that with our basic example from the get started section. We’ll take our simple prompt + model chain, which under the hood already defines a lot of functionality, and see what it would take to recreate all of it.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# model = ChatOpenAI(model="gpt-3.5-turbo")
from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

## Invoke

In the simplest case, we just want to pass in a topic string and get back a joke string:

Without LCEL

In [2]:
from typing import List

prompt_template = "Tell me a short joke about {topic}"
# client = openai.OpenAI()

def call_chat_model(messages: List[dict]) -> str:
    response = model.generate(messages)
    return response

def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    print(prompt_value)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

invoke_chain("cats")

Tell me a short joke about cats


LLMResult(generations=[[Generation(text='Why do cats always land on their feet? Because they have nine lives and still have to prove it! 🐱😅')]], llm_output=None, run=[RunInfo(run_id=UUID('749f61c4-3562-43fa-a96d-6f693bbac857'))])

## LECL

In [3]:
from langchain.schema.runnable import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
# model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

"Why don't scientists trust atoms? Because they make up everything!"

## Stream

If we want to stream results instead, we’ll need to change our function:
Without LCEL

In [4]:
from typing import Iterator


def stream_chat_model(messages: List[dict]) -> Iterator[str]:
    stream = model.generate(messages, stream=True)
    return stream.generations[0][0].text
"""     for response in stream:
        content = response
        if content is not None:
            yield content """

def stream_chain(topic: str) -> Iterator[str]:
    prompt_value = prompt.format(topic=topic)
    return stream_chat_model([{"role": "user", "content": prompt_value}])

# stream_chain("ice cream")

for chunk in stream_chain("ice cream"):
    print(chunk, end="", flush=True)



Why don't scientists trust atoms? Because they make up everything!

LECL

In [5]:
""" for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True) """
for chunk in chain.stream("ice cream"):
    print(chunk)

code
msg
data
success


In [6]:
chain.batch(["ice cream", "spaghetti", "dumplings"])

["Why don't scientists trust atoms? Because they make up everything!",
 "Why don't scientists trust spaghetti? Because it's always pulling pasta tricks! 😄",
 'Why did the dumpling win an award?因为它成功地“包裹”了最佳口感！']

In [7]:
# chain.ainvoke("ice cream")
# chain.abatch_invoke(["ice cream", "ice cream", "ice cream"])
response =await chain.ainvoke("ice cream")
response

"Why don't scientists trust atoms? Because they make up everything!"

In [8]:
chain.batch(["猫","狗","猪"])

['猫：喵，给你讲一个关于猫的笑话。\\n\\n有一天，一只猫走进了一家汽车制造厂，对工程师说：“我想买一辆会爬树的汽车。”\\n\\n工程师惊讶地问：“会爬树的汽车？你确定吗？”\\n\\n猫回答：“当然，不然我买它干嘛？”\\n\\n工程师想了想，说：“那你等着，我给你定制一辆。”\\n\\n一周后，工程师把一辆带有很多利爪的汽车交给了猫。猫非常高兴，付完钱后就开着新车离开了。\\n\\n第二天，猫回到工厂，满脸愤怒地对工程师说：“骗子！你说这车会爬树，结果它只会挖洞！”\\n\\n工程师一脸无辜地回答：“但它不是给你定制了一辆带有很多利爪的汽车吗？”\\n\\n猫无语地走了，心想：“谁说猫一定会爬树呢？”\\n\\n哈哈，喵，你觉得这个笑话有趣吗？',
 '一个人走进了宠物店，问老板：“这只狗多少钱？”老板回答：“这是 Wirehaired Pointing Griffon，它需要一点时间来适应新环境，所以价格是其他的狗的两倍。”那个人问：“为什么要这么贵？”老板回答：“你没看见它后面的那个牌子吗？写着‘友情提醒：主人是个瞎子。’”',
 '一个人问：“猪是怎么上树的？” 另一个人回答：“变成烤猪蹄就上去了。”\\n\\n（注：这个笑话是在调侃猪一般来说不会上树，但经过烹饪加工后的烤猪蹄却可以挂在树上。）']

## Parallelize steps

RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

In [9]:
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

from langchain_core.runnables.base import RunnableParallel
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "python"})

{'joke': "Why don't scientists trust atoms?\\n\\nBecause they make up everything!",
 'poem': 'Python, sleek and divine,\\nSlithering through codeine.'}